# WASOC Jupyterlite Demo notebook

The below is basic testing of the standard functionality available in the jupyterlite notebooks environment.

## Saving useful notebooks

Once you have built a useful notebook it can be worth saving it back to a github repository, to do this just navigate to a repository (such as [wasoc-notebook](https://github.com/wagov/wasoc-notebook)) and upload the `.ipynb` file. It is sensible to clear all outputs and check over your notebook before uploading to make sure there are no sensitive credentials, if using the `/proxy` endpoints in this environment properly, it should be straightforward to ensure creds are never saved locally or in notebooks.

## Upgrading this jupyterlite instance

As the instance uses a lot of caching and service workers in the background, content may become stale at times. The below approach is simplest to clear all the data and force a reload, however it will delete local notebooks saved in your browser (make sure to download them or save to repositories first).

- Open [chrome://settings/siteData](chrome://settings/siteData) or [edge://settings/siteData](edge://settings/siteData)
- Type this sites hostname on the Search input.
- Click on the trash bin to delete cached data and cookies for that web site (includes service workers and local storage).

In [ ]:
import piplite, js
await piplite.install([
    "pandas",
    f"{js.origin}/jupyter/jlitepkgs-0.1.3-py3-none-any.whl"
])
import pandas as pd
from jlitepkgs import http_call

## Loading data from sentinel using KQL
kql2df will run a global query and return all relevant data as a pandas Dataframe

In [ ]:
df = pd.DataFrame(http_call("api/v2/query",method="POST", data="""
SecurityIncident
| summarize count() by Severity, Status, ProviderName
| order by count_
""").json())
df.head()

## Loading data from an API using http_call
The [proxy](https://github.com/wagov/siem-query-utils/blob/main/siem_query_utils/proxy.py) endpoint passes request back to origins, optionally with embedded credentials, while attempting to strip any sensitive headers and cookies that may reveal sessions or credentials between origins.

- [/proxy/apis](/proxy/apis) will list all apis available with saved authentication. Some tools to build new configs are available at [/proxy/docs](/proxy/docs)
- API's with credentials saved in the `/proxy` session are able to be queried easily using `http_call`.
- API's don't need credentials but must be in session to be accessed via the `/proxy` endpoint.

In [ ]:
# List all saved apis
http_call("apis").json()

In [ ]:
# Credentialed api example
http_call("abuseipdb_v2/check", params={"ipAddress": "8.8.8.8"}).json()

In [ ]:
# Uncredentialed api example
http_call("httpbin/get").json()